<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EA%B8%B0%EC%82%AC_3%ED%9A%8C_%EC%9E%91%EC%97%85%ED%98%952_%EC%97%AC%ED%96%89%EC%9E%90%EB%B3%B4%ED%97%98%EA%B0%80%EC%9E%85%EC%97%AC%EB%B6%80%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 79.8kB/s]


In [3]:
!kaggle datasets download -d tejashvi14/travel-insurance-prediction-data

  0% 0.00/13.0k [00:00<?, ?B/s]
100% 13.0k/13.0k [00:00<00:00, 11.6MB/s]


In [4]:
!unzip /content/travel-insurance-prediction-data.zip

Archive:  /content/travel-insurance-prediction-data.zip
  inflating: TravelInsurancePrediction.csv  


In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def exam_data_load(df, target, id_name="", null_name=""):

    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/TravelInsurancePrediction.csv")
df = df.drop(columns='Unnamed: 0', axis=1)
X_train, X_test, y_train, y_test = exam_data_load(df, target='TravelInsurance')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print()
print("X_train")
print(X_train.head())
print()
print(X_train.describe())
print()
print(X_train.info())
print()
print(X_train.isna().sum())
print()
cols_train = X_train.columns
for col in cols_train:
    print(X_train[col].value_counts())
    print()

print()
print("X_test")
print(X_test.head())
print()
print(X_test.describe())
print()
print(X_test.info())
print()
print(X_test.isna().sum())
print()
cols_test = X_test.columns
for col in cols_test:
    print(X_test[col].value_counts())
    print()

object_cols = []
for col in cols_train:
    if X_train[col].dtype == 'object':
        object_cols.append(col)
print("object 열")
print(object_cols)
print()
LE = LabelEncoder()
for col in object_cols:
    X_train[col] = LE.fit_transform(X_train[col])
    X_test[col] = LE.transform(X_test[col])

X_train = X_train.drop(columns='id')
X_test_id = X_test['id']
X_test = X_test.drop(columns='id')

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=2022)
model.fit(X_train, y_train['TravelInsurance'])
print("모델 점수 train data:", model.score(X_train, y_train['TravelInsurance']))
prediction = model.predict(X_test)
print("모델 점수 test data:", model.score(X_test, prediction))
print()
output = pd.DataFrame({"id" : X_test_id, "TravelInsurance" : prediction})
print(output.head())
output.to_csv('20220620.csv', index=False)

(1589, 9) (398, 9) (1589, 2) (398, 2)

X_train
      id  Age               Employment Type GraduateOrNot  AnnualIncome  \
659  659   34  Private Sector/Self Employed           Yes        500000   
779  779   25  Private Sector/Self Employed           Yes       1400000   
643  643   28  Private Sector/Self Employed           Yes       1100000   
386  386   28  Private Sector/Self Employed           Yes       1350000   
700  700   28  Private Sector/Self Employed           Yes       1450000   

     FamilyMembers  ChronicDiseases FrequentFlyer EverTravelledAbroad  
659              3                0            No                  No  
779              4                0            No                 Yes  
643              5                1            No                  No  
386              5                0            No                 Yes  
700              4                0           Yes                 Yes  

                id          Age  AnnualIncome  FamilyMembers  Chronic